In [49]:
# coding: utf-8
# наше всё
import numpy as np
import pandas as pd

#настройки pandas, с которыми лучше почти всегда
pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)


# будем отображать графики прямо в jupyter'e
import seaborn as sns
import matplotlib.pyplot as plt
# лучше не делать в линуксе
# графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 12, 9

In [2]:
# footbal stat without api
# http://wildstat.ru/p/50/ch/EUR_CC_1988_1989

# дорогой и мощный api
# https://footystats.org/download-stats-csv

# каталог всего
# https://www.jokecamp.com/blog/guide-to-football-and-soccer-data-and-apis/

In [37]:
# хелп тут: https://www.football-data.org/documentation/quickstart
# полный список соревнований (за все годы)
url='http://api.football-data.org/v2/competitions'
# параметры конкретного соревнования (за все годы) 
# в примере испания
# url='http://api.football-data.org/v2/competitions/2014'
# спиоск матчей конкретного соревнования, тут уже есть фильтры
# в примере испания
# url='http://api.football-data.org/v2/competitions/2014/matches'
# данные по конкретному матчу
# севилья - райо вельйкано
# url='http://api.football-data.org/v2/matches/331356'
# данные команды
# url='http://api.football-data.org/v2/teams/78'

In [38]:
import requests
headers = {
    'X-Auth-Token': '44ce50aab7504f17ae93e674300009f6',
}

params = (
#     ('matchday', '1'),
#     ('dateFrom','2021-08-14'),
#     ('dateTo','2021-08-16'),
#     ('stage','REGULAR_SEASON'),
#     ('status','FINISHED'),
#     ('season','2016'),
)

r = requests.get(url, 
                 params=params, headers=headers)

print(r.status_code)

200


In [40]:
r.json().keys()

dict_keys(['count', 'filters', 'competitions'])

In [53]:
l1=[]
l2=[]
for i in range(r.json()['count']):
    l1.append(r.json()['competitions'][i]['name'])
    l2.append(r.json()['competitions'][i]['id'])
    
tournaments_table=pd.DataFrame([l1, l2]).T
tournaments_table.columns=['Name', 'ID']

In [54]:
tournaments_table.sample(3)

,Name,ID
80,Serie C,2123
15,Supercoupe de Belgique,2010
120,Scottish Cup,2087


In [69]:
tournaments_table[tournaments_table['ID']==2014]

,Name,ID
131,Primera Division,2014


In [36]:
# на вход - id турнире, на выходе - json
def get_league_data(id):
    url='http://api.football-data.org/v2/competitions/'+str(id)+'/matches'
    import requests
    headers = {
        'X-Auth-Token': '44ce50aab7504f17ae93e674300009f6',
    }

    r = requests.get(url, headers=headers)

    print(r.status_code)
    return r.json()

# когда получил запрос со всей лигой, её можно перевести в готовый датафрейм
def get_league_match(an):
    match_id_lst=[]
    home_lst=[]
    away_lst=[]
    h_score_lst=[]
    aw_score_lst=[]
    status_lst=[]
    date_lst=[]

    for i in range(an['count']):
        match_id_lst.append(an['matches'][i]['id'])
        home_lst.append(an['matches'][i]['homeTeam']['name'])
        away_lst.append(an['matches'][i]['awayTeam']['name'])
        h_score_lst.append(an['matches'][i]['score']['fullTime']['homeTeam'])
        aw_score_lst.append(an['matches'][i]['score']['fullTime']['awayTeam'])
        status_lst.append(an['matches'][i]['status'])
        date_lst.append(an['matches'][i]['utcDate'])
        
    league_season_df=pd.DataFrame([
                                match_id_lst,
                                home_lst,
                                away_lst,
                                h_score_lst,
                                aw_score_lst,
                                status_lst,
                                date_lst
                                    ]).T

    league_season_df.columns=['id', 'home', 'away', 'gh', 'ga', 'status', 'date']
    
    return league_season_df

In [77]:
for id in tournaments_table['ID'].values:#[125:132]:
    try:
        an=get_league_data(id)
        league_season_df=get_league_match(an)
        print(tournaments_table[tournaments_table['ID']==id]['Name'].values[0])
    except:
        print(id, 'no acces')

403
2006 no acces
403
2023 no acces
429
2024 no acces
429
2025 no acces
429
2149 no acces
429
2147 no acces
429
2008 no acces
429
2026 no acces
429
2020 no acces
429
2012 no acces
429
2022 no acces
429
2027 no acces
429
2028 no acces
429
2033 no acces
429
2009 no acces
429
2010 no acces
429
2032 no acces
429
2034 no acces
429
2035 no acces
429
2037 no acces
429
2038 no acces
429
2036 no acces
429
2029 no acces
429
2013 no acces
429
2039 no acces
429
2040 no acces
429
2041 no acces
429
2042 no acces
429
2043 no acces
429
2048 no acces
429
2044 no acces
429
2045 no acces
429
2046 no acces
429
2047 no acces
429
2049 no acces
429
2051 no acces
429
2050 no acces
429
2141 no acces
429
2052 no acces
429
2139 no acces
429
2056 no acces
429
2055 no acces
429
2053 no acces
429
2054 no acces
429
2030 no acces
429
2016 no acces
429
2021 no acces
429
2057 no acces
429
2146 no acces
429
2001 no acces
429
2154 no acces
429
2018 no acces
429
2007 no acces
429
2058 no acces
429
2059 no acces
429
2031 n

In [72]:
len(league_season_df)

380

In [35]:
league_season_df

,id,home,away,gh,ga,status,date
0,331355,Valencia CF,Getafe CF,1,0,FINISHED,2021-08-13T19:00:00Z
1,331358,Cádiz CF,Levante UD,1,1,FINISHED,2021-08-14T17:30:00Z
2,331359,RCD Mallorca,Real Betis Balompié,1,1,FINISHED,2021-08-14T17:30:00Z
3,331357,CA Osasuna,RCD Espanyol de Barcelona,0,0,FINISHED,2021-08-14T20:00:00Z
4,331362,Deportivo Alavés,Real Madrid CF,1,4,FINISHED,2021-08-14T20:00:00Z
...,...,...,...,...,...,...,...
375,330988,Real Madrid CF,Real Betis Balompié,None,None,SCHEDULED,2022-05-22T18:00:00Z
376,330989,CA Osasuna,RCD Mallorca,None,None,SCHEDULED,2022-05-22T18:00:00Z
377,330990,Granada CF,RCD Espanyol de Barcelona,None,None,SCHEDULED,2022-05-22T18:00:00Z
378,330991,FC Barcelona,Villarreal CF,None,None,SCHEDULED,2022-05-22T18:00:00Z


In [67]:
import Levenshtein as lev
api_team_lst=league_season_df['home'].value_counts().index

for team in api_team_lst:
    distance_lst=[]
    for ch in spain_check_lst:
        d=lev.distance(team, ch)
        distance_lst.append(d)
    print(min(distance_lst))
    ind=np.argmin(distance_lst)
    print(team, '-', spain_check_lst[ind])

9
Deportivo Alavés - Deportivo La Coruna
0
Villarreal CF - Villarreal CF
0
Getafe CF - Getafe CF
1
Levante UD - Levante Ud
0
FC Barcelona - FC Barcelona
7
RCD Espanyol de Barcelona - Espanyol Barcelona
9
Real Betis Balompié - Real Betis
0
CA Osasuna - CA Osasuna
0
Elche CF - Elche CF
3
Real Madrid CF - Real Madrid
9
Club Atlético de Madrid - Atletico Madrid
10
Real Sociedad de Fútbol - Real Sociedad
0
Valencia CF - Valencia CF
5
Athletic Club - Athletic Bilbao
0
RCD Mallorca - RCD Mallorca
0
Granada CF - Granada CF
1
Cádiz CF - Cadiz CF
0
Sevilla FC - Sevilla FC
10
Rayo Vallecano de Madrid - Rayo Vallecano
6
RC Celta de Vigo - Celta Vigo


In [57]:
spain_check_lst=['FC Barcelona', 'Real Madrid', 'Valencia CF', 'Athletic Bilbao',
       'Atletico Madrid', 'Espanyol Barcelona', 'Sevilla FC', 'Real Sociedad',
       'Real Zaragoza', 'Real Betis', 'Celta Vigo', 'Deportivo La Coruna',
       'Real Valladolid', 'Sporting Gijon', 'Racing Santander', 'CA Osasuna',
       'Malaga CF', 'Real Oviedo', 'Ud Las Palmas', 'RCD Mallorca',
       'Villarreal CF', 'Granada CF', 'Rayo Vallecano', 'Elche CF',
       'Hercules CF', 'Real Murcia', 'Getafe CF', 'CD Tenerife', 'Levante Ud',
       'Cd Alaves', 'Cadiz CF', 'Ce Sabadell', 'Ud Salamanca', 'CD Logrones',
       'Cd Castellon', 'Burgos CF', 'Cordoba CF', 'Albacete', 'Ud Almeria',
       'Sd Eibar', 'Recreativo Huelva', 'Pontevedra CF', 'Sd Compostela',
       'CD Numancia', 'Arenas De Getxo', 'Gimnastic', 'CD Leganes',
       'Cd Alcoyano', 'Real Jaen', 'CF Extremadura', 'Merida Ad', 'Girona FC',
       'Real Union', 'Ad Almeria', 'Ce Europa', 'Xerez Cd', 'SD Huesca',
       'Lleida Esportiu', 'Atletico Tetuan', 'CD Condal', 'CD Leonesa',
       'Ue Lleida', 'Real Madrid Castilla']